In [ ]:
# Setup

from requests import get, post, put, delete
from blowfish import Cipher
from ast import literal_eval

url = "http://192.168.0.10:5000"

def get_cipher_key(token: str) -> bytes:
    data = {
        "token": token
    }

    key = get(url + "/key", json=data).text
    return str.encode(key)


def get_iv(token: str) -> bytes:
    data = {
        "token": token
    }

    iv = get(url + "/iv", json=data).text
    return str.encode(iv)


def encrypt_title(cipher: Cipher, title: str, iv: str) -> str:
    encrypted_title = b"".join(cipher.encrypt_cfb(title, iv))

    return str(encrypted_title)

def decrypt_title(cipher: Cipher, encrypted_title: str, iv: str) -> str:
    encrypted_title = literal_eval(encrypted_title)
    decrypted_title = b"".join(cipher.decrypt_cfb(encrypted_title, iv))

    return decrypted_title.decode()

In [ ]:
# Demonstration - Note Creation

def main():
    token = input("Token: ").strip().lower()
    title = str.encode(input("Title: ").strip().lower())
    
    if not token or not title:
        print("Please enter an account token and note title.")
        return

    key = get_cipher_key(token)
    iv = get_iv(token)
    cipher = Cipher(key)
    
    encrypted_title = encrypt_title(cipher, title, iv)
    
    data = {
        "token": token,
        "title": encrypted_title
    }
    
    response = post(url + "/notes", json=data)

    print(response.text)

main()

In [ ]:
# Demonstration - Notes Retrieval

def main():
    token = input("Token: ").strip().lower()

    if not token:
        print("Please enter an account token.")
        return

    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return
    
    key = get_cipher_key(token)
    iv = get_iv(token)
    cipher = Cipher(key)

    response = get(url + "/notes", json=data)

    for note in response.json():
        id = note["id"]
        title = note["title"]
        title = decrypt_title(cipher, title, iv).title()
        modified = note["modified"]
        author = note["author"].title()

        print("")
        
        print(f"Note ID: {id}")
        print(f"Title: {title}")
        print(f"Updated: {modified}")
        

main()

In [ ]:
# Demonstration - Updating a Note's Title

def main():
    token = input("Token: ").strip().lower()

    if not token:
        print("Please enter a token.")
        return
    
    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return

    key = get_cipher_key(token)
    iv = get_iv(token)
    cipher = Cipher(key)
    
    note_id = int(input("Note ID: ").strip().lower())

    if not note_id:
        print("Please enter a note ID.")
        return

    title = str.encode(input("New Title: "))

    data = {
        "note_id": note_id,
        "token": token,
        "title": encrypt_title(cipher, title, iv)
    }

    response = put(url + "/notes", json=data)
    print(response.text)

main()

In [ ]:
# Demonstration - Note Deletion

def main():
    token = input("Token: ").strip().lower()

    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return

    note_id = int(input("Note ID: "))

    if not note_id:
        print("Please enter a note ID number.")
        return

    data = {
        "token": token,
        "note_id": note_id
    }

    response = delete(url + "/notes", json=data)
    print(response.text)

main()